In [1]:
import numpy as np
import astropy.io.fits as pyfits
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS

from sys import argv
import warnings
import time
import os

In [2]:
#---------------------------------------
# Input some parameters from parameter.ipynb
#---------------------------------------
%run parameter.ipynb

#---------------------------------------
# Input parameters used in this program:

###-------------------------------------
### Path name & outname & imageName
# path_image, path_BT, path_figure, path_samping
# outname_b4,  outname_b7
# BT_B4, BT_B7, BT_slt_image, BT_div_image
# Mask_image, Mask_4A1_image , Mask_4A2_image

###-----------
# criteria and rms for the imaging
# [Jy beam-1]
# rms_I_b4, rms_Q_b4, rms_PI_b4, I_clip_b4, PI_lowclip_b4, PI_highclip_b4,
# rms_I_b7, rms_Q_b7, rms_PI_b7, I_clip_b7, PI_lowclip_b7, PI_highclip_b7
# sampling all, sampling_b4, sampling_b7

###-----------
# some parameters about the targets
# dist [pc]
# ra_4A1, ra_4A2, ra_4A1_4A2
# dec_4A1, dec_4A2, dec_4A1_4A2 

/usr/local/lib/python2.7/site-packages/matplotlib/cbook/deprecation.py:107: MatplotlibDeprecationWarning: The mpl_toolkits.axes_grid module was deprecated in version 2.1. Use mpl_toolkits.axes_grid1 and mpl_toolkits.axisartist provies the same functionality instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [3]:
#----------------------------------------
# define the functions
#----------------------------------------
def nyquist_sampling(outname, I_clip, PI_lowclip, PI_highclip, \
                     rms_I, rms_Q, rms_PI, sampling_interval, Select_region):
    
    ###------------------------------    
    # Get the Imagename
    I_imageName, Q_imageName, U_imageName, PI_imageName, PI_cut3PI_imageName,\
    PA_imageName, Per_imageName, Per_cut3PI_imageName, BT_imageName = slt_imageName(outname)
           
    ###------------------------------
    # Get the FITS and convert them from 4 to 2 dimensions (y, x)
    I_data = Getdata(I_imageName)
    Q_data = Getdata(Q_imageName)
    U_data = Getdata(U_imageName)
    PI_data = Getdata(PI_imageName)
    PA_data = Getdata(PA_imageName)
    Per_data = Getdata(Per_imageName)
    BT_data = Getdata(BT_imageName)
    BT_slt_data = Getdata(BT_slt_imageName)
    BT_div_data = Getdata(BT_div_imageName) 
    Mask_data = Getdata(Mask_imageName)
    Mask_4A1_data = Getdata(Mask_4A1_imageName)
    Mask_4A2_data = Getdata(Mask_4A2_imageName)  
    
    # Get the header
    I_hd = pyfits.getheader(I_imageName)

    # Fitsfile have 2 dimension: (y, x)
    Ny, Nx = I_data.shape
    nx, ny = np.meshgrid(np.arange(Nx), np.arange(Ny))
    
    # Get the coordinates (of world coor. system) of Stokes I intensity map
    wcs = WCS(I_hd)
    Coord = SkyCoord.from_pixel(nx, ny, wcs)
    RA_data = Coord.ra.deg
    Dec_data = Coord.dec.deg     
    
    ###------------------------------
    # Set the different criteria
    # standard criteria
    crit_stand = (I_data > I_clip) &\
                (PI_data > PI_lowclip) &\
                (PI_data < PI_highclip) &\
                (np.isnan(Per_data) == False) &\
                (nx % sampling_interval == 0) &\
                (ny % sampling_interval == 0)
    
    # Specific criterion: split the 4A1 & 4A2
    # select data within the specific B4 contour (4A1 & 4A2)
    crit_mask = Mask_data > 0    
    # select 4A1 only
    crit_mask_4A1 = Mask_4A1_data > 0
    # select 4A2 only
    crit_mask_4A2 = Mask_4A2_data > 0
    
    ###------------------------------
    # Criteria selection
    if Select_region == 'mask':
        crit = crit_stand & crit_mask
    elif Select_region == 'mask_4A1':
        crit = crit_stand & crit_mask_4A1
    elif Select_region == 'mask_4A2':
        crit = crit_stand & crit_mask_4A2
    else:
        crit = crit_stand
    
    ###------------------------------
    # Get the selected pixel value based on the criteria
    I_crit_data = I_data[np.where(crit)]               # Stokes I [Jy beam-1]
    Q_crit_data = Q_data[np.where(crit)]               # Stokes Q [Jy beam-1]
    U_crit_data = U_data[np.where(crit)]               # Stokes U [Jy beam-1]
    PI_crit_data = PI_data[np.where(crit)]             # Polarized intensity [Jy beam-1]
    PA_crit_data = PA_data[np.where(crit)]             # Polarization angle [deg]
    Per_crit_data = Per_data[np.where(crit)]           # Polarization percentage [%]
    BT_crit_data = BT_data[np.where(crit)]             # Brightness temperature [K]
    BT_slt_crit_data = BT_slt_data[np.where(crit)]     # Brightness temperature [K]
    BT_div_crit_data = BT_div_data[np.where(crit)]     # Brightness temperature ratio [NA]
    RA_crit_data = RA_data[np.where(crit)]             # RA [deg]
    Dec_crit_data = Dec_data[np.where(crit)]           # Dec [deg]
    
    # Get the uncertainties of PI, PA, and Per
    PI_err_crit_data = PI_err(Q_crit_data, U_crit_data, rms_Q, rms_Q)
    PA_err_crit_data = PA_err(Q_crit_data, U_crit_data, rms_Q, rms_Q)
    Per_err_crit_data = Per_err(I_crit_data, Q_crit_data, U_crit_data, \
                              rms_I, rms_Q, rms_Q, \
                              PI_err_crit_data)
    Per_err_crit_data = np.where(Per_err_crit_data>0.1, Per_err_crit_data, 0.1)
    
    
    return I_crit_data, Q_crit_data, U_crit_data, \
            PI_crit_data, PA_crit_data, Per_crit_data, \
            PI_err_crit_data, PA_err_crit_data, Per_err_crit_data, \
            BT_crit_data, BT_slt_crit_data, BT_div_crit_data, \
            RA_crit_data, Dec_crit_data

#---------------------------------------
def Getdata(input_filename):
    # Input file is CASA format file with 4 dimension: (Stokes, freq, y, x)
    # Drop the dimension from 4 to 2: remain (y, x)    
    output_fits = pyfits.getdata(input_filename)[0][0]
    
    return output_fits

#---------------------------------------
def PI_err(Q, U, Q_err, U_err):
    part1 = np.power(Q,2)*np.power(Q_err,2) + np.power(U,2)*np.power(U_err,2)
    part2 = np.power(Q,2) + np.power(U,2)
    return np.sqrt(np.divide(part1, part2))

def PA_err(Q, U, Q_err, U_err):
    part1 = np.power(Q,2)*np.power(U_err,2) + np.power(U,2)*np.power(Q_err,2)
    part2 = np.power(np.power(Q,2) + np.power(U,2) , 2)
    return 0.5*np.sqrt(np.divide(part1, part2))

def Per_err(I, Q, U, I_err, Q_err, U_err, PI_err):
    part1 = np.divide(np.power(PI_err,2), np.power(I,2))
    part2 = np.divide(np.power(I_err,2)*(np.power(Q,2) + np.power(U,2)), np.power(I,4))
    return np.sqrt(part1 + part2)*100

#---------------------------------------
def GetTxtfile(outname, Select_region, SavetxtName, isMrt):
    
    # Get the rms & clip critera from the different outname
    rms_I, rms_Q, rms_PI, I_clip, PI_lowclip, PI_highclip, \
    sampling, PI_vmax, convert_unit = slt_parm(outname)
            
    # Get the data
    I_crit_array, Q_crit_array, U_crit_array, \
    PI_crit_array, PA_crit_array, Per_crit_array, \
    PI_err_crit_array, PA_err_crit_array, Per_err_crit_array, \
    BT_crit_array, BT_slt_crit_array, BT_div_crit_array, \
    RA_crit_array, Dec_crit_array = \
    nyquist_sampling(outname, I_clip, PI_lowclip, PI_highclip, \
                     rms_I, rms_Q, rms_PI, sampling, Select_region)
    
    if isMrt == True:
        #Save the data to a txt file for machine readable table
        # - array_lens = np.shape(RA_crit_array)[0]
        # - type_arr = (j*np.ones((array_lens,), dtype=np.int))
        np.savetxt(SavetxtName, \
                   np.column_stack((RA_crit_array, Dec_crit_array,\
                                    I_crit_array, Q_crit_array, U_crit_array, \
                                    PI_crit_array, PA_crit_array, PA_err_crit_array, \
                                    Per_crit_array, Per_err_crit_array)),\
                   fmt='%.6f & %.6f & %.2g & %.2g & %.2g & %.2g & %.f & %.2g & %.2g & %.2g')
        
    else:  
        # Save the data to a txt file for data analysis
        np.savetxt(SavetxtName, \
                   np.column_stack((RA_crit_array, Dec_crit_array,\
                                    I_crit_array, Q_crit_array, U_crit_array,\
                                    PI_crit_array, PA_crit_array, Per_crit_array,\
                                    PI_err_crit_array, PA_err_crit_array, Per_err_crit_array,\
                                    BT_crit_array, BT_slt_crit_array, BT_div_crit_array,\
                                   )),\
                   fmt='%s & %s & %.3g & %.3g & %.3g & %.3g & %.3g & %.3g & %.3g & %.3g & %.3g & %.3g & %.3g & %.3g \\\\')
        
#---------------------------------------
def cal_dist(ra, dec, ra_target, dec_target):
    dist_target = np.sqrt(np.power(ra_target - ra, 2) + np.power(dec_target - dec, 2))
    return dist_target

In [4]:
# Star measure time
start_time = time.time()
#---------------------------------------
# Output the txt files from the observations

# Decide the input outname
outnames = [
             outname_b4, outname_b7
            ]

#----------------------------------------
for outname in outnames:

    #---------------------------------------------------------
    # All the selected pixel (4A1 & 4A2)

    # for data analysis
    Select_region = 'all'
    SavetxtName_all = '%snyquist_%s_all.txt'%(path_samping, outname) 
    GetTxtfile(outname, Select_region, SavetxtName_all, isMrt=False) 

    # machine readable table (mrt)
    SavetxtName_all_mrt = '%snyquist_%s_all_mrt.txt'%(path_samping, outname)
    GetTxtfile(outname, Select_region, SavetxtName_all_mrt, isMrt=True)  

    #---------------------------------------------------------
    # Split the 4A1 and 4A2

    # (1) 4A1 & 4A2: only the values within the masked B7 contour
    isMrt = False
    Select_region = 'mask'
    SavetxtName_mask = '%snyquist_%s_mask.txt'%(path_samping, outname)
    GetTxtfile(outname, Select_region, SavetxtName_mask, isMrt)  

    # (2) 4A1 only
    Select_region = 'mask_4A1'
    SavetxtName_mask_4A1 = '%snyquist_%s_mask_4A1.txt'%(path_samping, outname)
    GetTxtfile(outname, Select_region, SavetxtName_mask_4A1, isMrt)      

    # (3) 4A2 only
    Select_region = 'mask_4A2'
    SavetxtName_mask_4A2 = '%snyquist_%s_mask_4A2.txt'%(path_samping, outname)
    GetTxtfile(outname, Select_region, SavetxtName_mask_4A2, isMrt) 

    #---------------------------------------------------------
    # Inner100 AU: Only the values within certain radius

    # Set the radius (e.g., 100au)
    radius_au = 100
    arcsec2deg = np.divide(1., 60*60)      # 1 arcsec [deg]
    AU2deg = np.divide(arcsec2deg, dist)   # 1 AU [deg]
    radius_deg = radius_au*AU2deg                # radius [AU in deg]

    # Get the data from textfile (previous step)
    nq_mask_array = np.array(np.loadtxt(SavetxtName_mask, dtype = str))
    nq_mask_4A1_array = np.array(np.loadtxt(SavetxtName_mask_4A1, dtype = str))
    nq_mask_4A2_array = np.array(np.loadtxt(SavetxtName_mask_4A2, dtype = str))

    # Obtain the ra & dec from mask txt 
    ra_mask_4A1_array = nq_mask_4A1_array.T[0].astype(np.float)
    dec_mask_4A1_array = nq_mask_4A1_array.T[2].astype(np.float)
    ra_mask_4A2_array = nq_mask_4A2_array.T[0].astype(np.float)
    dec_mask_4A2_array = nq_mask_4A2_array.T[2].astype(np.float)

    # Calculate the distance between each point(pixel) and 4A1/4A2
    dist_mask_4A1 = cal_dist(ra_mask_4A1_array, dec_mask_4A1_array, ra_4A1, dec_4A1)
    dist_mask_4A2 = cal_dist(ra_mask_4A2_array, dec_mask_4A2_array, ra_4A2, dec_4A2)

    # (4) innermost 100 AU region of IRAS4A1   
    nq_mask_4A1_inner100au_array = nq_mask_4A1_array[np.where(dist_mask_4A1 < radius_deg)]

    # (5) innermost 100 AU region of IRAS4A2  
    nq_mask_4A2_inner100au_array = nq_mask_4A1_array[np.where(dist_mask_4A2 < radius_deg)]

    # Set the txt filename
    SavetxtName_mask_4A1_inner100au = '%snyquist_%s_mask_4A1_inner100au.txt'%(path_samping, outname)
    SavetxtName_mask_4A2_inner100au = '%snyquist_%s_mask_4A2_inner100au.txt'%(path_samping, outname)

    # Save the txtfile
    np.savetxt(SavetxtName_mask_4A1_inner100au, nq_mask_4A1_inner100au_array, fmt="%s")
    np.savetxt(SavetxtName_mask_4A2_inner100au, nq_mask_4A2_inner100au_array, fmt="%s")

    #---------------------------------------------------------
    # Inner100 AU: Only the values within certain radius

    # Difference: (mask - innermost 100au)
    nq_mask_4A1_tuple = [tuple(i) for i in nq_mask_4A1_array]
    nq_mask_4A2_tuple = [tuple(i) for i in nq_mask_4A2_array]
    nq_mask_4A1_inner100au_tuple = [tuple(i) for i in nq_mask_4A1_inner100au_array]
    nq_mask_4A2_inner100au_tuple = [tuple(i) for i in nq_mask_4A1_inner100au_array]

    # (6) IRAS4A1: nearby region without inner 100au
    nq_mask_4A1_wo_inner100au_set = set(nq_mask_4A1_tuple) - set(nq_mask_4A1_inner100au_tuple)        
    nq_mask_4A1_wo_inner100au_array = np.array(list(nq_mask_4A1_wo_inner100au_set))      

    # (7) IRAS4A2: nearby region without inner 100au
    nq_mask_4A2_wo_inner100au_set = set(nq_mask_4A2_tuple) - set(nq_mask_4A2_inner100au_tuple)
    nq_mask_4A2_wo_inner100au_array = np.array(list(nq_mask_4A2_wo_inner100au_set))

    # Set txt filename
    SavetxtName_mask_4A1_wo_inner100au = '%snyquist_%s_mask_4A1_wo_inner100au.txt'%(path_samping, outname)
    SavetxtName_mask_4A2_wo_inner100au = '%snyquist_%s_mask_4A2_wo_inner100au.txt'%(path_samping, outname)

    # Save the txtfile
    np.savetxt(SavetxtName_mask_4A1_wo_inner100au, nq_mask_4A1_wo_inner100au_array, fmt = '%s')
    np.savetxt(SavetxtName_mask_4A2_wo_inner100au, nq_mask_4A2_wo_inner100au_array, fmt = '%s')

#---------------------------------------------------------
# End measure time
elapsed_time = time.time() - start_time
print ('#----------------------------------------')
print ("Done! Time spending = ", elapsed_time, "seconds.")

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in greater
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in less
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:47: RuntimeWarning: invalid value encountered in greater
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:49: RuntimeWarning: invalid value encountered in greater
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:51: RuntimeWarning: invalid value encountered in greater


#----------------------------------------
('Done! Time spending = ', 37.83867907524109, 'seconds.')
